In [2]:
#initialize data
import numpy as np
with open('data/data_20_800.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True)
i = 50
p = data[i]['processing_times'] 
d = data[i]['due_dates']
print(data[i]['optimal_solution'])

732


In [3]:
#edd sort
def edd_sort(p,d):
    indexes = np.arange(len(d))
    combined = zip(indexes, p,d)
    sorted_combined = sorted(combined, key=lambda x: x[2])
    indexes_sorted, p_sorted, d_sorted = zip(*sorted_combined)
    indexes, p, d = np.array(indexes_sorted), np.array(p_sorted),np.array(d_sorted)
    return indexes, p, d

In [4]:
def tardiness(schedule, processing_times, due_dates):
    current_time = 0
    tardiness = 0
    for job in schedule:
        current_time += processing_times[job]
        tardiness += max(0, current_time - due_dates[job])
    return tardiness

In [5]:
#Dynamic Programming
from functools import lru_cache
from functools import cache
@cache
def calculate_tardiness(next_time, due_time):
    return max(0, next_time - due_time)

@cache
def dp(time, remaining_jobs_bitmask):
    if remaining_jobs_bitmask == 0:
        return 0, []  # Return total tardiness and the sequence of jobs

    min_tardiness = float('inf')
    optimal_sequence = []

    for job_index in range(len(p)):
        if remaining_jobs_bitmask & (1 << job_index):
            next_time = time 
            tardiness = calculate_tardiness(next_time, d[job_index])
            new_tardiness, sequence = dp(next_time- p[job_index], remaining_jobs_bitmask & ~(1 << job_index))

            total_tardiness = new_tardiness + tardiness
            if total_tardiness < min_tardiness:
                min_tardiness = total_tardiness
                optimal_sequence = sequence + [job_index]  # Append current job to the optimal subsequence

    return min_tardiness, optimal_sequence

dp.cache_clear()
calculate_tardiness.cache_clear()
initial_time = sum(p) #start from the last job
initial_bitmask = (1 << len(p)) - 1  # all jobs are available initially

# Solve the problem
min_tardiness, optimal_sequence = dp(initial_time, initial_bitmask)

print("Minimum total tardiness:", min_tardiness)
print("Optimal job sequence:", optimal_sequence)


Minimum total tardiness: 732
Optimal job sequence: [19, 16, 14, 11, 9, 8, 7, 6, 0, 5, 2, 4, 17, 13, 15, 10, 1, 3, 12, 18]


In [6]:
tardiness(optimal_sequence, p, d)

732

In [10]:
#vanillaMIP
#calisan model bu- no VI
import time
import gurobipy as gp
from gurobipy import GRB
def MIP(p,d):
    st1= time.time()
    N = len(p)
    M = sum(p)  # a large number for big-M constraints

    #model init
    model = gp.Model("SingleMachineScheduling")
    model.setParam('TimeLimit',60*60)

    # Decision variables
    u = model.addVars(N, N, vtype=GRB.BINARY, name="u")  # u[j, k]-binary var to denote assignment
    ck = model.addVars(N, vtype=GRB.CONTINUOUS, name="ck")  # completion time at position k
    Cj = model.addVars(N, vtype=GRB.CONTINUOUS, name="Cj")  # completion time of job j
    T = model.addVars(N, vtype=GRB.CONTINUOUS, name="T")  # tardiness of job j

    # Constraints
    # Each job is assigned to exactly one position
    model.addConstrs((u.sum(j, '*') == 1 for j in range(N)), name="AssignJobToOnePosition")

    # Each position is assigned to exactly one job
    model.addConstrs((u.sum('*', k) == 1 for k in range(N)), name="AssignPositionToOneJob")

    # Completion time of the first job
    model.addConstr(ck[0] == gp.quicksum(p[j] * u[j, 0] for j in range(N)), name="C1")

    # Completion time relationships
    model.addConstrs(
        (ck[k] >= ck[k - 1] + gp.quicksum(p[j] * u[j, k] for j in range(N)) for k in range(1, N)),
        name="CompletionTimeRelations",
    )


    # Constraint set (12): Completion time constraints
    # u == 1 -> Cj(job completion) is g/e than ck(position completion)
    # u == 0 ->  no bound 
    model.addConstrs(
        (Cj[j] >= ck[k] - M * (1 - u[j, k]) for j in range(N) for k in range(N)), name="CompletionTimeConstraints"
    )

    # Constraint set (13): Define tardiness
    model.addConstrs((T[j] >= Cj[j] - d[j] for j in range(N)), name="DefineTardiness")

    # Non-negativity constraints (14, 15, 16)
    model.addConstrs((T[j] >= 0 for j in range(N)), name="TNonNegativity")
    model.addConstrs((Cj[j] >= 0 for j in range(N)), name="CjNonNegativity")
    model.addConstrs((ck[k] >= 0 for k in range(N)), name="ckNonNegativity")

    # Objective: Minimize total tardiness
    model.setObjective(gp.quicksum(T[j] for j in range(N)), GRB.MINIMIZE)
    # Optimize the model
    model.optimize()

    # Display results
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found")
        for j in range(N):
            print(f"Job {j}: Tardiness = {T[j].X}")


    # Check if the model found an optimal solution
    if model.status == GRB.OPTIMAL:

        print("Optimal solution found.")
        for j in range(N):
            print(f"Job {j}: Tardiness = {T[j].X}")
        # Calculate total tardiness
        total_tardiness = sum(T[j].X for j in range(N))
        print(f"Total Tardiness: {total_tardiness}")

        # Determine the optimal job sequence
        optimal_sequence = []
        for k in range(N):
            for j in range(N):
                if u[j, k].X == 1:
                    optimal_sequence.append(j)

        print("Optimal job sequence:", optimal_sequence)

        # Display tardiness for each job in the optimal sequence
        for k, job in enumerate(optimal_sequence):
            print(f"Position {k}: Job {job}, Tardiness = {T[job].X}")
    else:
        print("No optimal solution found.")
    e1 = time.time()
    return e1-st1, optimal_sequence
time, opts = MIP(p, d)

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))



CPU model: 13th Gen Intel(R) Core(TM) i9-13900F, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 540 rows, 460 columns and 2539 nonzeros
Model fingerprint: 0x1ff3f514
Variable types: 60 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 185 rows and 20 columns
Presolve time: 0.01s
Presolved: 355 rows, 440 columns, 2084 nonzeros
Variable types: 0 continuous, 440 integer (400 binary)
Found heuristic solution: objective 2841.0000000
Found heuristic solution: objective 1805.0000000

Root relaxation: objective 0.000000e+00, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    

In [14]:
#just one VI
import time
import gurobipy as gp
from gurobipy import GRB
def VI(p,d,indexes):
    
    st= time.time()
    # Sample data: Define number of jobs (N), processing times, and due dates
    N = len(p)
    M = sum(p)  # a large number for big-M constraints
    p_sorted, d_sorted, indexes_sorted = zip(*sorted(zip(p, d,indexes)))
    p = np.array(p_sorted)
    d = np.array(d_sorted)
    indexes = np.array(indexes_sorted)
    

    pi_jk = [[0 for _ in range(N)] for _ in range(N)]
    ro_jk = [[0 for _ in range(N)] for _ in range(N)]

    for j in range(N):
        for k in range(N):
            if k <= j:
                pi_jk[j][k] = sum(p[:k-1])
            else:
                pi_jk[j][k] = sum(p[:k]) - p[j]

    for j in range(N):
        for k in range(N):
            if k >= j:
                ro_jk[j][k] = sum(p[k+1:])
            else:
                ro_jk[j][k] = sum(p[k:j-1]) + sum(p[j+1:])

    a_jl = [[0 for _ in range(N)] for _ in range(N)]
    for j in range(N):
        for l in range(N):
            if l<k:
                a_jl[j][l] = ro_jk[j][N-k+l]
            elif l>k:
                a_jl[j][l] = p[j] + pi_jk[j][l-k]


    # Create a new model
    model = gp.Model("SingleMachineScheduling")
    model.setParam('TimeLimit',60*60)

    # Decision variables
    u = model.addVars(N, N, vtype=GRB.BINARY, name="u")  # u[j, k]
    ck = model.addVars(N, vtype=GRB.CONTINUOUS, name="ck")  # completion time at position k
    Cj = model.addVars(N, vtype=GRB.CONTINUOUS, name="Cj")  # completion time of job j
    T = model.addVars(N, vtype=GRB.CONTINUOUS, name="T")  # tardiness of job j

    # Constraints
    # Constraint set (8): Each job is assigned to exactly one position
    model.addConstrs((u.sum(j, '*') == 1 for j in range(N)), name="AssignJobToOnePosition")

    # Constraint set (9): Each position is assigned to exactly one job
    model.addConstrs((u.sum('*', k) == 1 for k in range(N)), name="AssignPositionToOneJob")

    # Constraint set (10): Completion time of the first job
    model.addConstr(ck[0] == gp.quicksum(p[j] * u[j, 0] for j in range(N)), name="C1")

    # Constraint set (11): Completion time relationships
    model.addConstrs(
        (ck[k] >= ck[k - 1] + gp.quicksum(p[j] * u[j, k] for j in range(N)) for k in range(1, N)),
        name="CompletionTimeRelations",
    )

    # Constraint set (12): Completion time constraints
    model.addConstrs(
        (Cj[j] >= ck[k] - M * (1 - u[j, k]) for j in range(N) for k in range(N)), name="CompletionTimeConstraints"
    )

    # Constraint set (13): Define tardiness
    model.addConstrs((T[j] >= Cj[j] - d[j] for j in range(N)), name="DefineTardiness")

    # Non-negativity constraints (14, 15, 16)
    model.addConstrs((T[j] >= 0 for j in range(N)), name="TNonNegativity")
    model.addConstrs((Cj[j] >= 0 for j in range(N)), name="CjNonNegativity")
    model.addConstrs((ck[k] >= 0 for k in range(N)), name="ckNonNegativity")

    # Objective: Minimize total tardiness
    model.setObjective(gp.quicksum(T[j] for j in range(N)), GRB.MINIMIZE)


    # Add the valid inequalities
    model.addConstrs(
        (Cj[j] >= p[j] + gp.quicksum(pi_jk[j][k] * u[j, k] for k in range(1, N)) for j in range(N)),
        name="ValidInequality",
    )

    # Optimize the model
    model.optimize()



    # Results
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found.")

        for j in range(N):
            print(f"Job {j}: Tardiness = {T[j].X}")

        # Calculate total tardiness
        total_tardiness = sum(T[j].X for j in range(N))
        print(f"Total Tardiness: {total_tardiness}")

        # Determine the optimal job sequence
        optimal_sequence = []
        for k in range(N):
            for j in range(N):
                if u[j, k].X == 1:
                    optimal_sequence.append(j)

        print("Optimal job sequence:", optimal_sequence)

        # Display tardiness for each job in the optimal sequence
        for k, job in enumerate(optimal_sequence):
            print(f"Position {k + 1}: Job {job}, Tardiness = {T[job].X}")
    else:
        print("No optimal solution found.")
    e = time.time()

    neww= indexes[optimal_sequence]
    return e-st, neww
indexes = np.arange(len(d))
VI(p, d, indexes)

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900F, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 560 rows, 460 columns and 2919 nonzeros
Model fingerprint: 0x4a8e1555
Variable types: 60 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 185 rows and 20 columns
Presolve time: 0.01s
Presolved: 375 rows, 440 columns, 2171 nonzeros
Variable types: 0 continuous, 440 integer (400 binary)
Found heuristic solution: objective 3255.0000000

Root relaxation: objective 3.260000e+02, 223 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

(0.2846066951751709,
 array([19, 10,  6,  2,  7, 16, 11,  5,  4,  0, 14,  9,  8, 17, 13, 15,  1,
         3, 12, 18]))

In [ ]:
tardiness(opts, p, d)